In [2]:
from tinkoff.invest.utils import now
import os
from datetime import timedelta
from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.constants import INVEST_GRPC_API, INVEST_GRPC_API_SANDBOX
from configs import tink_API_key, company_figi

TOKEN = tink_API_key
prices = []

with Client(TOKEN, target=INVEST_GRPC_API_SANDBOX) as client:
    for candle in client.get_all_candles(
        figi=company_figi["SIBN"],
        from_=now() - timedelta(minutes=120),
        interval=CandleInterval.CANDLE_INTERVAL_1_MIN,
    ):
        prices.append(candle.close.units)

In [31]:
from tinkoff.invest.schemas import OrderType, OrderDirection

In [4]:
import numpy as np

mean_price = np.mean(prices)
std = np.std(prices)

In [23]:
std_steps = np.arange(0.5, 2.01, 0.5)
enter_prices = []
qty = []
for i in range(len(std_steps)):
    enter_prices.append(mean_price - std * std_steps[i])
    qty.append(i + 1)
enter_prices = np.round(enter_prices, 1)
qty = np.array(qty, dtype=np.float32)

total_sum = (qty * enter_prices).sum()
print("Требуется денег: ", total_sum)

Требуется денег:  8049.599999999999


In [33]:
with Client(TOKEN, target=INVEST_GRPC_API) as client:
    client.orders.post_order(
        figi=company_figi["SIBN"], price=200, quantity=2, order_type=1
    )  # , direction=OrderDirection(1))

49d5e08bdf70b98fe8f21cc7c0d63d35 PostOrder INVALID_ARGUMENT 30019


RequestError: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30019', Metadata(tracking_id='49d5e08bdf70b98fe8f21cc7c0d63d35', ratelimit_limit='300, 300;w=60', ratelimit_remaining=299, ratelimit_reset=57, message="missing parameter: 'direction'"))

In [10]:
from tinkoff.invest import Client, LimitOrderRequest, OrderDirection, Quotation

TOKEN = "your_api_token_here"
FIGI = "figi_of_the_instrument"
QUANTITY = 10  # Number of shares to buy
PRICE = 100.0  # Price at which you want to buy the shares

# Start by creating a client instance
with Client(TOKEN) as client:
    # Create the limit order request
    # Note that the price must be specified in the smallest currency unit (kopecks, cents, etc.)
    # Thus, it must be transformed into `Quotation`
    order_response = client.orders.post_order(
        figi=FIGI,
        quantity=QUANTITY,
        price=Quotation(units=int(PRICE), nano=int((PRICE - int(PRICE)) * 1e9)),
        direction=OrderDirection.ORDER_DIRECTION_BUY,
        account_id="your_account_id",
        order_type=OrderType.ORDER_TYPE_LIMIT,
        order_id="your_unique_order_id",
    )

    # Check the response
    if order_response:
        print("Order placed successfully")
        print(f"Order ID: {order_response.order_id}")
    else:
        print("Failed to place the order")

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [12]:
np.arange(0, 10, 0.5)

array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ,
       6.5, 7. , 7.5, 8. , 8.5, 9. , 9.5])

In [6]:
std

2.288708206652497